## 한글 문서의 분류
올리브영으로부터 크롤링한 향수 리뷰를 이용하여 분류 연습.
향수 리뷰와 향수 이름을 학습해서 주어진 리뷰내용으로 어떤 향수에 대한 리뷰인지를 예측하고자 함.

### data file 내용
여러 향수에 대한 약 1000개의 리뷰
csv 파일 안에 리뷰내용, 가격, 향수이름의 순으로 저장되어 있음

In [1]:
import csv

text = []
y = []
with open('C:/Users/Seo/perfume7.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[0]) #영화 리뷰를 text 리스트에 추가
            y.append(row[2]) #영화이름을 text 리스트에 추가

In [2]:
print('Num of samples: {}'.format(len(text)))
print('perfume of reivews: {}'.format(set(y)))

Num of samples: 973
perfume of reivews: {'나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', 'Name', '클린 웜코튼 EDP 30ml (N) 기프트세트', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml'}


In [3]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0)
# 비율을 지정하지 않으면 75:25로 분할됨

In [4]:
len(X_train) #1827의 0.75

729

In [5]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

In [6]:
print(twitter_tag.morphs(X_train[1])) #둘째 리뷰에 대해 형태소 단위로 tokenize

['클', '림', '심플', '?', '향', '을', '구매', '하고', '샘플', '로', '온', '향', '을', '동생', '이', '맡아', '봤는데', '너무', '마음', '에', '들어', '해서', '선물', '로', '사줬습니다', '.', '호불호', '없이', '비누', '향', '좋아하시는', '분', '들이라면', '좋아할', '향', '이에요', '.', '여름', '보다는', '지금', '처럼', '추운', '계절', '에', '잘', '어울리는', '포근한', '비누', '향', '입니다', '.']


In [7]:
twitter_tag.nouns(X_train[1]) #둘째 리뷰에서 명사만 추출

['림',
 '심플',
 '향',
 '구매',
 '샘플',
 '온',
 '향',
 '동생',
 '마음',
 '선물',
 '호불호',
 '비누',
 '향',
 '분',
 '향',
 '여름',
 '지금',
 '계절',
 '비누',
 '향']

In [8]:
def twit_tokenizer(text): # Twitter 형태소 분석기의 명사추출함수를 tokenizer 함수로 사용
    return twitter_tag.nouns(text)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2) #Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
# twit_tokenizer 대신 twitter_tag.nouns를 직접 써도 됨
# 하나의 문서에서만 출현한 단어는 쓸모가 없으므로 제외, 즉 최소 document frequency를 2로 설정

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

clf = LogisticRegression() # logistic regression 분류기 선언
clf.fit(X_train_tfidf, y_train) # 분류기 학습
print('Train score', clf.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', clf.score(X_test_tfidf, y_test)) # test data 예측정확도
print(X_train_tfidf.shape) # 총 979개의 명사로 이루어짐

Train score 0.9039780521262003
Test score 0.7049180327868853
(729, 979)


In [10]:
X_test[:10] #test data에서 앞 10개를 출력

['정말 고민 엄청 많이 하다가 저번에 기획세트때 구매를 놓치고 계속 아쉬워하다가 이번에 올영세일하면서 살짝 추가 세일하길래 이때다 하고 쿠폰까지 알뜰하게 받아서 구매하게 되었는데 정말..왜 이제서야 구매했나 싶어요  정말 너무 제 취향의 항수여서 행복하네요ㅠㅠ 원래 밖에 잘 안 나가는데 이거 뿌리고 싶어서 나가고 싶을 정도에요ㅋㅋ  저는 중성적인 향을 좋아하는 편인데 이 향수는 약간 시원하면서 살짝 달달한 그런 느낌의 향수인데(설명 못하는 편) 데일리하게 사용하기에 좋고 패키지도 하얀색으로 깔끔하고 좋아요!( 대신 뚜껑을 닫을때 조금 힘주면서 닫아야합니다!)  내일 나갈일 있는데 이거 뿌리고 나가야지.. 요놈 덕분에 나갈때 신이나네요!!!너무 좋아!!!!!',
 '언니가 사용중인데, 스칠때마다 은은하게 나는 향이 좋아서 저는  언니꺼 하나 뺏고,  마침 지인이 생일이라 선물했습니다. 향이 너무 좋다고 좋아라네요~ 향수도 많이 가지고 있는데, 은은하게 내고 싶을때 사용하면 좋을꺼 같아요.. 지속력이 향수만큼은 아니라 가지고 다니면서 손 씻고 나서 조금씩 팔목에 바르면 좋을꺼 같아요.',
 '남여상관업이너무죠은향같아요.. 이걸죠아하게된동기가조금웃깃데ㅎㅎ 생각나서..구입해서..계속쓰게될꺼같네요ㅎㅎ',
 '과대 광고에 속은것 같아요 기내에서 무슨향이냐고 물어볼만큼 좋진 않던데  차라리 록시땅이 백만배 낫지',
 '향 너무 좋아요 남친이랑 같이 쓰고있어요 남녀 무관 다 사용가능',
 '매장에서 테스트 해보고 뿌린 후에 향이 너무 좋아 집에서도 생각나서 인터넷으로 바로 구매한 제품입니다. 향이 처음엔 강하게 느낄 수도 있는데 시간이 지날수록 은은하고 계속 맡고 싶은 향입니다. 지속력은 그리 오래가지는 않지만 그래도 다쓰면 또 구매하고 싶은 상품입니다',
 '처음 써보는 브랜드 향수라 고민하다가 송혜교가 쓴다고 추천 받아서 덜컥 사버렸던 향수입니다. 파우더리한 머스크 향기가 넘 매력있어요.',
 '향도 고급스럽고 시향해서 샀는데 문제는 지속력이 로션수준도 안된다는것.다른

In [11]:
clf.predict(X_test_tfidf[:10]) # test data의 앞 10개에 대한 예측내용

array(['클린 웜코튼 EDP 30ml (N) 기프트세트', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml',
       '클린 웜코튼 EDP 30ml (N) 기프트세트', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml',
       '클린 웜코튼 EDP 30ml (N) 기프트세트', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml',
       '나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '클린 웜코튼 EDP 30ml (N) 기프트세트',
       '나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '클린 웜코튼 EDP 30ml (N) 기프트세트'],
      dtype='<U30')

In [12]:
print(y_test[:10]) # test data 앞 10개의 실제 향수이름

['나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml', '클린 웜코튼 EDP 30ml (N) 기프트세트', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml', '나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '유즈 솔리드 퍼퓸 002 스테이포에버 30ml', '나르시소 로드리게즈 퓨어 머스크 포 허 EDP 30ml', '클린 웜코튼 EDP 30ml (N) 기프트세트']


### 성능을 개선하기 위한 노력

In [13]:
# morphs()는 명사 외에도 모든 형태소를 포함
print(twitter_tag.morphs(X_train[1]))

['클', '림', '심플', '?', '향', '을', '구매', '하고', '샘플', '로', '온', '향', '을', '동생', '이', '맡아', '봤는데', '너무', '마음', '에', '들어', '해서', '선물', '로', '사줬습니다', '.', '호불호', '없이', '비누', '향', '좋아하시는', '분', '들이라면', '좋아할', '향', '이에요', '.', '여름', '보다는', '지금', '처럼', '추운', '계절', '에', '잘', '어울리는', '포근한', '비누', '향', '입니다', '.']


In [14]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2) # 명사 대신 모든 형태소를 사용
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#명사만 사용한 것에 비해 train score는 상승, test score도 상승

Train score 0.9108367626886146
Test score 0.7172131147540983
(729, 2375)


In [15]:
print(twitter_tag.pos(X_train[1], norm=True, stem=True)) #pos()는 형태소와 품사를 함께 제공

[('크다', 'Verb'), ('림', 'Noun'), ('심플', 'Noun'), ('?', 'Punctuation'), ('향', 'Noun'), ('을', 'Josa'), ('구매', 'Noun'), ('하고', 'Josa'), ('샘플', 'Noun'), ('로', 'Josa'), ('온', 'Noun'), ('향', 'Noun'), ('을', 'Josa'), ('동생', 'Noun'), ('이', 'Josa'), ('맡다', 'Verb'), ('보다', 'Verb'), ('너무', 'Adverb'), ('마음', 'Noun'), ('에', 'Josa'), ('들다', 'Verb'), ('하다', 'Verb'), ('선물', 'Noun'), ('로', 'Josa'), ('사주다', 'Verb'), ('.', 'Punctuation'), ('호불호', 'Noun'), ('없이', 'Adverb'), ('비누', 'Noun'), ('향', 'Noun'), ('좋아하다', 'Adjective'), ('분', 'Noun'), ('들이다', 'Verb'), ('좋아하다', 'Adjective'), ('향', 'Noun'), ('이에요', 'Josa'), ('.', 'Punctuation'), ('여름', 'Noun'), ('보다는', 'Josa'), ('지금', 'Noun'), ('처럼', 'Josa'), ('추다', 'Verb'), ('계절', 'Noun'), ('에', 'Josa'), ('자다', 'Verb'), ('어울리다', 'Verb'), ('포근하다', 'Adjective'), ('비누', 'Noun'), ('향', 'Noun'), ('이다', 'Adjective'), ('.', 'Punctuation')]


In [16]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
#            result.append('/'.join([word, tag]))
    return result

In [17]:
print(twit_tokenizer2(X_train[1])) # 사용 예

['크다', '림', '심플', '향', '구매', '샘플', '온', '향', '동생', '맡다', '보다', '마음', '들다', '하다', '선물', '사주다', '호불호', '비누', '향', '좋아하다', '분', '들이다', '좋아하다', '향', '여름', '지금', '추다', '계절', '자다', '어울리다', '포근하다', '비누', '향', '이다']


In [18]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2) #명사, 동사, 형용사를 이용하여 tfidf 생성
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# 현재까지 중에서 train score와 test score가 가장 뛰어남

Train score 0.9204389574759945
Test score 0.7213114754098361
(729, 1351)


In [19]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 하면?
def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [20]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#성능이 오히려 떨어지고 품사 표시 없이 전체를 다 사용한 경우에 비해 train은 떨어지고, test는 올라감

Train score 0.9039780521262003
Test score 0.7254098360655737
(729, 1766)


In [21]:
# train score가 높으므로 ridge를 쓰면 어떨까?
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# train score가 많이 올라가는 현상이 벌어짐
# test score가 떨어짐

Train set score: 0.992
Test set score: 0.697


In [22]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

Train set score: 0.759
Test set score: 0.705
Used features count: 106 out of 1766


In [23]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=239, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_) 
print(svd.components_.shape)

[0.01272797 0.01535934 0.01391181 0.0119879  0.00981152 0.00938478
 0.00888663 0.00824467 0.00785292 0.0076218  0.00739273 0.00718091
 0.00698807 0.00690575 0.00665107 0.00638571 0.0063371  0.00617013
 0.00601059 0.00594473 0.00576946 0.00568242 0.00562685 0.00559611
 0.0053678  0.0053023  0.00526049 0.0051909  0.00509188 0.00504222
 0.00502261 0.00492295 0.00487171 0.00482909 0.0046877  0.00467147
 0.00460281 0.00454153 0.00450718 0.00444192 0.00440306 0.00436811
 0.00426428 0.00424517 0.00421482 0.00418513 0.00414284 0.00407579
 0.00404944 0.00398521 0.00395656 0.00393727 0.00388572 0.00378201
 0.00376215 0.00375652 0.00370826 0.00369295 0.00365433 0.0036443
 0.00358839 0.00355951 0.00351737 0.00348155 0.0034484  0.00344025
 0.00342797 0.00337484 0.00336    0.00329511 0.00328196 0.00326446
 0.00322843 0.00321604 0.00320984 0.00318434 0.00314877 0.00313846
 0.00310071 0.00305728 0.00304596 0.00303976 0.00301643 0.00298961
 0.00296337 0.00294752 0.00293947 0.00290358 0.00287307 0.00286

In [24]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

Train set score: 0.860
Test set score: 0.738


In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=twit_tokenizer2, min_df=2).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

Train set dimension: (729, 1351)
Test set dimension: (244, 1351)
Train set score: 0.888
Test set score: 0.758
